<a href="https://colab.research.google.com/github/z-gard/analysis/blob/main/notebooks/public_school_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 学校
- [小学校（学級数、教員数、職員数、住所等）](https://catalog.data.metro.tokyo.lg.jp/dataset/t000021d1700000002/resource/cbfc36fa-1cd1-4e5a-9773-de48647f4d18)
- [中学校（学級数、教員数、職員数、住所等）](https://catalog.data.metro.tokyo.lg.jp/dataset/t000021d1700000002/resource/73e248a7-8ebd-45a4-83aa-357229d3584d)
- [幼稚園（教員数、職員数、住所等）](https://catalog.data.metro.tokyo.lg.jp/dataset/t000021d1700000002/resource/7906a64c-bd69-4c65-a17c-2b9cb95bc0e5)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [136]:
import os
import requests
import json
import pandas as pd
from urllib.parse import urlparse, quote
import io
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [2]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [44]:
def get_data_csv_url(url, encoding='utf-8', excel=False):
    res = requests.get(url)
    if res.status_code == 200:
        if not excel:
            return pd.read_csv(io.BytesIO(res.content), encoding=encoding)
        else:
            return pd.read_excel(io.BytesIO(res.content))
    else:
        raise Exception(f'status_code: {res.status_code} ({url})')

In [90]:
# 公立小学校
els_url = 'http://www.kyoiku.metro.tokyo.jp/administration/statistics_and_research/list_of_public_school/files/school_lists2017_csv/houkokushokouritsu01_20.csv'
# 公立中学校
jhs_url = 'http://www.kyoiku.metro.tokyo.jp/administration/statistics_and_research/list_of_public_school/files/school_lists2017_csv/houkokushokouritsu02_03.csv'
# 公立幼稚園
kin_url = 'http://www.kyoiku.metro.tokyo.jp/administration/statistics_and_research/list_of_public_school/files/school_lists2017_csv/houkokushokouritsu01_17.csv'

In [92]:
def get_school_info(url, school_type):
    df_school = get_data_csv_url(url, encoding='shift-jis')
    print(df_school.shape)
    df_school = df_school.rename(columns={'園名': '学校名'})
    df_school = df_school[['学校番号', '設置者', '学校名', '住所']]
    df_school = df_school[df_school['設置者'].notna()]
    df_school = df_school[(df_school['設置者'].str.endswith('区'))|(df_school['設置者'].str.endswith('都'))].reset_index(drop=True)
    df_school['施設分類'] = school_type
    df_school['正式名称'] = df_school['設置者'] + '立' + df_school['学校名'] + df_school['施設分類']
    return df_school

### 公立小学校

In [93]:
df_elementary_school = get_school_info(els_url, '小学校')
print(df_elementary_school.shape)

(1277, 20)
(823, 6)


In [94]:
df_elementary_school.head()

,学校番号,設置者,学校名,住所,施設分類,正式名称
0,201150,千代田区,麹町,麹町２−８,小学校,千代田区立麹町小学校
1,201160,千代田区,九段,富士見１−１−６,小学校,千代田区立九段小学校
2,201170,千代田区,番町,六番町８,小学校,千代田区立番町小学校
3,201180,千代田区,富士見,富士見１−１０−３,小学校,千代田区立富士見小学校
4,201190,千代田区,お茶の水,猿楽町１−１−１,小学校,千代田区立お茶の水小学校


### 公立中学校

In [82]:
df_juniorhigh_school = get_school_info(jhs_url, '中学校')
print(df_juniorhigh_school.shape)

(615, 17)
(378, 6)


In [87]:
# 神田一橋・通信制を除外
df_juniorhigh_school = df_juniorhigh_school.drop_duplicates(subset='住所').reset_index(drop=True)
print(df_juniorhigh_school.shape)

(377, 6)


In [89]:
df_juniorhigh_school.head()

,学校番号,設置者,学校名,住所,施設分類,正式名称
0,301010,千代田区,麹町,平河町２−５−１,中学校,千代田区立麹町中学校
1,301060,千代田区,神田一橋,一ツ橋２−６−１４,中学校,千代田区立神田一橋中学校
2,302030,中央区,晴海,晴海１−５−３,中学校,中央区立晴海中学校
3,302040,中央区,日本橋,東日本橋１−１０−１,中学校,中央区立日本橋中学校
4,302050,中央区,銀座,銀座８−１９−１５,中学校,中央区立銀座中学校


### 公立幼稚園

In [95]:
df_kindergarten = get_school_info(kin_url, '幼稚園')
print(df_kindergarten.shape)

(172, 13)
(164, 6)


In [99]:
df_kindergarten.head()

,学校番号,設置者,学校名,住所,施設分類,正式名称
0,101150,千代田区,麹町,麹町２−８,幼稚園,千代田区立麹町幼稚園
1,101160,千代田区,九段,富士見１−１−６,幼稚園,千代田区立九段幼稚園
2,101170,千代田区,番町,六番町８,幼稚園,千代田区立番町幼稚園
3,101190,千代田区,お茶の水,猿楽町１−１−１,幼稚園,千代田区立お茶の水幼稚園
4,101200,千代田区,千代田,神田司町２−１６,幼稚園,千代田区立千代田幼稚園


### 公立学校

In [100]:
df_public_school = pd.concat([
    df_elementary_school,
    df_juniorhigh_school,
    df_kindergarten
]).reset_index(drop=True)
print(df_public_school.shape)

(1364, 6)


## 場所を探す

In [108]:
YOUR_API_KEY = 'xxxxxx'

In [137]:
def get_place_info(place_name, unique_no, is_rating=True):
    place = quote(place_name)
    rating = '%2Crating' if is_rating else ''
    
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place}&inputtype=textquery&fields=name{rating}%2Cgeometry&key={YOUR_API_KEY}"
    response = requests.get(url)
    if response.status_code == requests.codes.ok:
        try:
            _df = pd.DataFrame(response.json()['candidates'])
            _df['no'] = unique_no
            _df['place_name'] = place_name
            _df['lat'] = _df['geometry'].apply(lambda x: x['location']['lat'])
            _df['lng'] = _df['geometry'].apply(lambda x: x['location']['lng'])
            if not is_rating:
                _df['rating'] = np.NaN
            return _df[['no', 'place_name', 'name', 'lat', 'lng', 'rating']]
        except Exception as e:
            print(unique_no, place_name, e)
            pass
    else:
        print(unique_no, place_name, response.status_code)
    
    return pd.DataFrame([{'no': unique_no, 'place_name': place_name}])

In [116]:
df_result = pd.DataFrame()
for index, row in df_public_school.iterrows():
    _df = get_place_info(row['正式名称'], row['学校番号'])
    df_result = pd.concat([df_result, _df])
df_result = df_result.reset_index(drop=True)
print(df_result.shape)

204040 新宿区立愛日小学校 "['rating'] not in index"
207070 墨田区立小梅小学校 "['rating'] not in index"
207360 墨田区立立花吾嬬の森小学校 "['rating'] not in index"
208300 江東区立第三砂町小学校 "['rating'] not in index"
211310 大田区立洗足池小学校 "['rating'] not in index"
217390 北区立西ケ原小学校 "['rating'] not in index"
219250 板橋区立板橋第九小学校 "['rating'] not in index"
221160 足立区立興本小学校 "['rating'] not in index"
311110 大田区立大森第七中学校 "['rating'] not in index"
313100 渋谷区立原宿外苑中学校 "['rating'] not in index"
315040 杉並区立阿佐ヶ谷中学校 "['rating'] not in index"
318010 荒川区立第一中学校 "['rating'] not in index"
319090 板橋区立志村第三中学校 "['rating'] not in index"
321220 足立区立渕江中学校 "['rating'] not in index"
103230 港区立青南幼稚園 "['rating'] not in index"
103240 港区立白金台幼稚園 "['rating'] not in index"
104160 新宿区立四谷第六幼稚園 "['rating'] not in index"
105120 文京区立千駄木幼稚園 "['rating'] not in index"
108080 江東区立辰巳幼稚園 "['rating'] not in index"
108230 江東区立大島幼稚園 "['rating'] not in index"
112010 世田谷区立塚戸幼稚園 "['rating'] not in index"
114010 中野区立かみさぎ幼稚園 "['rating'] not in index"
116020 豊島区立南長崎幼稚園 "['rating'] no

### データが取得できなかった学校

In [168]:
df_na_data = df_result[df_result['name'].isna()][['no', 'place_name']]
print(len(df_na_data))

27


In [139]:
# ratingなしで検索
df_result_2 = pd.DataFrame()
for index, row in df_na_data.iterrows():
    _df = get_place_info(row['place_name'], row['no'], is_rating=False)
    df_result_2 = pd.concat([df_result_2, _df])
df_result_2 = df_result_2.reset_index(drop=True)
print(df_result_2.shape)

(27, 6)


In [140]:
df_result_2

,no,place_name,name,lat,lng,rating
0,204040,新宿区立愛日小学校,区立愛日小学校,35.699717,139.733988,NaN
1,207070,墨田区立小梅小学校,墨田区立小梅小学校,35.714357,139.805834,NaN
2,207360,墨田区立立花吾嬬の森小学校,区立立花吾嬬の森 小学校,35.706544,139.829336,NaN
3,208300,江東区立第三砂町小学校,江東区立第三砂町小学校,35.672613,139.833334,NaN
4,211310,大田区立洗足池小学校,Senzokuike Elementary School,35.600080,139.686648,NaN
5,217390,北区立西ケ原小学校,北区立西ヶ原小学校,35.743218,139.736902,NaN
6,219250,板橋区立板橋第九小学校,板橋区立板橋第九小学校,35.754680,139.704263,NaN
7,221160,足立区立興本小学校,足立区立興本小学校,35.772948,139.773475,NaN
8,311110,大田区立大森第七中学校,区立大森第七中学校,35.578241,139.689343,NaN
9,313100,渋谷区立原宿外苑中学校,渋谷区立原宿外苑中学校,35.673486,139.705537,NaN


### 重複している検索結果

In [144]:
df_duplicate = df_result[df_result.duplicated(subset='place_name')]
print(df_duplicate.shape)

(39, 6)


In [161]:
# 緯度経度の差が大きいところのみ、確認
df_merge_duplicate = pd.merge(df_result, df_duplicate[['no', 'lat', 'lng']], on='no', how='inner', suffixes=['', '_2'])
df_merge_duplicate['lat_diff'] = (df_merge_duplicate['lat'] - df_merge_duplicate['lat_2']).abs()
df_merge_duplicate['lng_diff'] = (df_merge_duplicate['lng'] - df_merge_duplicate['lng_2']).abs()

In [163]:
df_merge_duplicate[(df_merge_duplicate['lat_diff'] > 0.001)|(df_merge_duplicate['lng_diff'] > 0.001)]

,no,place_name,name,lat,lng,rating,lat_2,lng_2,lat_diff,lng_diff
4,208280,江東区立砂町小学校,Sunamachi Elementary School,35.677544,139.829058,2.4,35.683168,139.823464,0.005624,0.005593
6,208400,江東区立北砂小学校,Kitasuna Elementary School,35.683168,139.823464,3.8,35.677544,139.829058,0.005624,0.005593
10,211400,大田区立萩中小学校,Haginaka Elementary School,35.549071,139.738651,3.8,35.550270,139.731756,0.001199,0.006895
12,211490,大田区立矢口小学校,Yaguchi Elementary School,35.562789,139.697869,3.8,35.568059,139.687117,0.005270,0.010752
16,214130,中野区立向台小学校,Nakano 1st Elementary School,35.695710,139.677580,5.0,35.691943,139.679607,0.003767,0.002027
22,217020,北区立王子第一小学校,Oji Daiichi Elementary School,35.764915,139.737764,3.0,35.763141,139.734273,0.001774,0.003490
28,219070,板橋区立志村第六小学校,Shimura Dairoku Elementary School,35.783735,139.680654,4.3,35.782457,139.680474,0.001278,0.000180
30,219390,板橋区立赤塚小学校,Akatsuka Elementary School,35.777459,139.639974,4.3,35.774331,139.643421,0.003128,0.003447
34,220350,練馬区立関町小学校,Sekimachi Elementary School,35.724978,139.571793,3.3,35.732591,139.573731,0.007613,0.001938
40,221330,足立区立島根小学校,Adachi Kuritsu Shimane Elementary School,35.780643,139.791858,4.1,35.782233,139.800676,0.001590,0.008818


### 結合

In [182]:
df_result_all = df_result.drop_duplicates(subset=['no', 'place_name'])
df_result_all = pd.concat([
    df_result_all[df_result_all['name'].notna()],
    df_result_2
])
print(df_result_all.shape)
assert len(df_result_all) == len(df_public_school)

(1364, 6)


In [254]:
df_public_school_master = pd.concat([
    df_public_school.set_index('学校番号'),
    df_result_all.set_index('no')[['lat', 'lng', 'rating']]
], axis=1).sort_index()
df_public_school_master.index.name='学校番号'
df_public_school_master = df_public_school_master.rename(columns={'lat':'緯度', 'lng':'経度', '正式名称':'名称', '住所':'所在地', '設置者':'市区町村名'}).reset_index()

In [255]:
df_public_school_master.head()

,学校番号,市区町村名,学校名,所在地,施設分類,名称,緯度,経度,rating
0,101150,千代田区,麹町,麹町２−８,幼稚園,千代田区立麹町幼稚園,35.685691,139.740164,0.0
1,101160,千代田区,九段,富士見１−１−６,幼稚園,千代田区立九段幼稚園,35.690666,139.741195,0.0
2,101170,千代田区,番町,六番町８,幼稚園,千代田区立番町幼稚園,35.687629,139.733426,5.0
3,101190,千代田区,お茶の水,猿楽町１−１−１,幼稚園,千代田区立お茶の水幼稚園,35.697445,139.748569,4.0
4,101200,千代田区,千代田,神田司町２−１６,幼稚園,千代田区立千代田幼稚園,35.693169,139.768274,0.0


In [256]:
# 設置者を市区町村名に変えたため、東京都から所在地の区に変更
df_public_school_master.loc[df_public_school_master['市区町村名'] == '東京都', '市区町村名'] = df_public_school_master['所在地'].str.extract('(^.*?[区市])', expand=False)

- 214130 中野区立向台小学校、214010 中野区立桃園小学校 -> 中野第一小学校 に統合
    - 向台小学校の名前を変更し、桃園小学校のレコードを削除

In [257]:
df_public_school_master[df_public_school_master['学校名'].isin(['向台', '桃園'])]

,学校番号,市区町村名,学校名,所在地,施設分類,名称,緯度,経度,rating
536,214010,中野区,桃園,本町３−１６−１,小学校,中野区立桃園小学校,35.695988,139.677294,3.2
545,214130,中野区,向台,弥生町１−２５−１,小学校,中野区立向台小学校,35.695710,139.677580,5.0


In [258]:
df_public_school_master['名称'] = df_public_school_master['名称'].str.replace('中野区立向台小学校', '中野区立第一小学校')
df_public_school_master['学校名'] = df_public_school_master['学校名'].str.replace('向台', '第一')
df_public_school_master = df_public_school_master.drop(index=[536]).reset_index(drop=True)    # 桃園小学校を削除

In [259]:
assert len(df_public_school_master[df_public_school_master['学校名'].isin(['向台', '桃園'])]) == 0

In [260]:
print(df_public_school_master.shape)
df_public_school_master

(1363, 9)


,学校番号,市区町村名,学校名,所在地,施設分類,名称,緯度,経度,rating
0,101150,千代田区,麹町,麹町２−８,幼稚園,千代田区立麹町幼稚園,35.685691,139.740164,0.0
1,101160,千代田区,九段,富士見１−１−６,幼稚園,千代田区立九段幼稚園,35.690666,139.741195,0.0
2,101170,千代田区,番町,六番町８,幼稚園,千代田区立番町幼稚園,35.687629,139.733426,5.0
3,101190,千代田区,お茶の水,猿楽町１−１−１,幼稚園,千代田区立お茶の水幼稚園,35.697445,139.748569,4.0
4,101200,千代田区,千代田,神田司町２−１６,幼稚園,千代田区立千代田幼稚園,35.693169,139.768274,0.0
...,...,...,...,...,...,...,...,...,...
1358,400010,台東区,白鴎高等学校附属,台東区元浅草３−１２−１２,中学校,東京都立白鴎高等学校附属中学校,35.707073,139.787558,3.6
1359,400030,墨田区,両国高等学校附属,墨田区江東橋１−７−１４,中学校,東京都立両国高等学校附属中学校,35.694523,139.810445,3.9
1360,400040,武蔵野市,武蔵高等学校附属,武蔵野市境４−１３−２８,中学校,東京都立武蔵高等学校附属中学校,35.708113,139.539029,3.9
1361,400050,中野区,富士高等学校附属,中野区弥生町５−２１−１,中学校,東京都立富士高等学校附属中学校,35.689595,139.667014,4.1


In [261]:
df_public_school_master.to_csv(os.path.join(DIR_NAME, 'public_school_master.csv'), index=False)